In [6]:

from CandidateModel import CandidateModel
from DefaultConfigOptions import *
from NDPopulation import NDPopulation
from PopulationGroup import Independents
from Timings import Timings


def create_model_and_population(ideology_bins: int, ideology_dim: int) -> (CandidateModel, NDPopulation):
    ideology_bins = 64
    hidden_ratio = 4
    n_hidden = hidden_ratio * ideology_bins * ideology_dim
    n_latent = ideology_bins * ideology_dim
    batch_size = 128
    learn_rate = .001

    model = CandidateModel(ideology_bins=ideology_bins,
                                     ideology_dim=ideology_dim,
                                     n_hidden=n_hidden,
                                     n_latent = n_latent,
                                     learn_rate= learn_rate)

    population_means = np.zeros(shape=(ideology_dim,))
    population_stddev = np.ones(shape=(ideology_dim,))
    pop = NDPopulation(population_means, population_stddev)
    return model, pop


timings = Timings()
def build_model(dim: int) -> bool:
    print("building model dim " + str(dim))
    m, p = create_model_and_population(64, dim)
    for i in range(100):
        voters = [p.unit_sample_voter() for i in range(6)]
        candidates = [Candidate("c", Independents, v.ideology, 0) for v in voters]
        winner = candidates[np.random.randint(0, 6)]
        for i in range(len(candidates)):
            m.add_sample_from_candidates(candidates[i], candidates[0:i-1], winner)

    with timings.time_block("build_model " + str(dim)):
        for i in range(100):
            m.train(128)

    print("complete:  dim " + str(dim))
    return True

from concurrent.futures import ThreadPoolExecutor

dims=[1,2,3,4]
with ThreadPoolExecutor() as executor:
    with timings.time_block("serial"):
        for dim in dims:
            build_model(dim)
    with timings.time_block("parallel"):
        futures = executor.map(build_model, dims)
        [print(f) for f in futures]

timings.print()

building model dim 1
complete:  dim 1
building model dim 2
complete:  dim 2
building model dim 3
complete:  dim 3
building model dim 4
complete:  dim 4
building model dim 1building model dim 2

building model dim 3
building model dim 4
complete:  dim 1
True
complete:  dim 2
True
complete:  dim 3
True
complete:  dim 4
True
       build_model 1 2   62.468 31.23412
       build_model 2 2   77.248 38.62398
       build_model 3 2   97.523 48.76146
       build_model 4 2  104.293 52.14629
              serial 1  106.746 106.74592
            parallel 1   66.674 66.67356
